# Dangerous Mutations

--------

## Clinical Disease Data Parsing

My manager comes to me Monday morning and says “I figured out our next step; we are going to pivot from an online craft store and become a data center for genetic disease information! I found **ClinVar** which is a repository that contains expert curated data, and it is free for the taking. This is a gold mine! Look at the file and tell me what gene and mutation combinations are classified as dangerous.”

In this project, I am going to extract the dangerous mutations, the final deliverable is a dataframe table to summarize these mutations with following information:

1) Gene name

2) Mutation ID number

3) Mutation Position (chromosome & position)

4) Mutation value (reference & alternate bases)

5) Clinical significance (CLNSIG)

6) Disease that is implicated

**Requirements**

1) Limit the output to the first 100 harmful mutations and tell my manager how many total harmful mutations were found in the file

2) Use the "clinvar_final.txt" data which could be downloaded from this link: https://drive.google.com/file/d/1Zps0YssoJbZHrn6iLte2RDLlgruhAX1s/view?usp=sharing 

3) Filter out junk and lines with no mutation data. Just focus on the data to deliver the result. 


Note:

* The unit of observation of this dataset is one row per mutation.

### VCF file description (Summarized from version 4.1)

```
* The VCF specification:

VCF is a text file format which contains meta-information lines, a header line, and then data lines each containing information about a position in the genome. The format also can contain genotype information on samples for each position.

* Fixed fields:

There are 8 fixed fields per record. All data lines are **tab-delimited**. In all cases, missing values are specified with a dot (‘.’). 

1. CHROM - chromosome number
2. POS - position DNA nuceleotide count (bases) along the chromosome
3. ID - The unique identifier for each mutation
4. REF - reference base(s)
5. ALT - alternate base(s)
6. FILTER - filter status
7. QUAL - quality
8. INFO - a semicolon-separated series of keys with values in the format: <key>=<data>

```
### Applicable INFO field specifications

```
GENEINFO = <Gene name>
CLNSIG =  <Clinical significance>
CLNDN = <Disease name>
```

### Sample ClinVar data (vcf file format - not exactly the same as the file to download!)

```
##fileformat=VCFv4.1
##fileDate=2019-03-19
##source=ClinVar
##reference=GRCh38							
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
1	949523	rs786201005	C	T	.	.	GENEINFO=ISG15;CLNSIG=5
1	949696	rs672601345	C	CG	.	.	GENEINFO=ISG15;CLNSIG=5;CLNDN=Cancer
1	949739	rs672601312	G	T	.	.	GENEINFO=ISG15;CLNDBN=Cancer
1	955597	rs115173026	G	T	.	.	GENEINFO=AGRN;CLNSIG=2; CLNDN=Cancer
1	955619	rs201073369	G	C	.	.	GENEINFO=AGG;CLNDN=Heart_dis 
1	957640	rs6657048	C	T	.	.	GENEINFO=AGG;CLNSIG=3;CLNDN=Heart_dis 
1	976059	rs544749044	C	T	.	.	GENEINFO=AGG;CLNSIG=0;CLNDN=Heart_dis 
```

In [11]:
## Load Raw Data
import pandas as pd
import io
import os
import re

pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# Load data to DataFrame, exclude any row start with #
with open('clinvar_final.txt', 'r') as fin:
    lines = [line for line in fin if not line.startswith('#')]
 
with open('clinvar_final.txt', 'r') as fin:
    headers = [line for line in fin if line.startswith('#')]
    

df = pd.read_csv(io.StringIO(''.join(lines)),
                 dtype={'CHROM': int, 'POS': str, 'ID': int, 'REF': str, 'ALT': str,
                        'FILTER': str,'QUAL': str, 'INFO': str},
                 sep='\t')
print(''.join(headers))
df.head(10)

#fileformat=VCFv4.1
#fileDate=2019-03-01
#source=ClinVar
#reference=GRCh38
#ID=<Description="ClinVar Variation ID">
#INFO=<ID=AF_ESP,Number=1,Type=Float,Description="allele frequencies from GO-ESP">
#INFO=<ID=AF_EXAC,Number=1,Type=Float,Description="allele frequencies from ExAC">
#INFO=<ID=AF_TGP,Number=1,Type=Float,Description="allele frequencies from TGP">
#INFO=<ID=ALLELEID,Number=1,Type=Integer,Description="the ClinVar Allele ID">
#INFO=<ID=CLNDN,Number=.,Type=String,Description="ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">
#INFO=<ID=CLNDNINCL,Number=.,Type=String,Description="For included Variant : ClinVar's preferred disease name for the concept specified by disease identifiers in CLNDISDB">
#INFO=<ID=CLNDISDB,Number=.,Type=String,Description="Tag-value pairs of disease database name and identifier, e.g. OMIM:NNNNNN">
##INFO=<ID=CLNDISDBINCL,Number=.,Type=String,Description="For included Variant: Tag-value pairs of disease databa

,CHROM,POS,ID,REF,ALT,FILTER,QUAL,INFO
0,1,1014O42,475283,G,A,.,.,AF_ESP=0.00546;AF_EXAC=0.00165;AF_TGP=0.00619;...
1,1,1O14122,542074,C,T,.,.,AF_ESP=0.00015;AF_EXAC=0.00010;ALLELEID=514926...
2,1,1014143,183381,C,T,.,.,"ALLELEID=181485;CLNDISDB=MedGen:C4015293,OMIM:..."
3,1,1014179,542075,C,T,.,.,"ALLELEID=514896;CLNDISDB=MedGen:C4015293,OMIM:..."
4,1,1014217,475278,C,T,.,.,AF_ESP=0.00515;AF_EXAC=0.00831;AF_TGP=0.00339;...
5,1,1014228,402986,G,A,.,.,AF_ESP=0.40158;AF_EXAC=0.37025;AF_TGP=0.33886;...
6,1,1014255,571208,G,A,.,.,"ALLELEID=556509;CLNDISDB=MedGen:C4015293,OMIM:..."
7,1,1014276,568195,G,A,.,.,"ALLELEID=556512;CLNDISDB=MedGen:C4015293,OMIM:..."
8,1,1014316,161455,C,CG,.,.,"ALLELEID=171289;CLNDISDB=MedGen:C4015293,OMIM:..."
9,1,1014359,161454,G,T,.,.,AF_EXAC=0.00001;ALLELEID=171288;CLNDISDB=MedGe...


In [12]:
## Extract Gene name, clincal significant and disease name from raw data

# the gene name is the value after the "GENEINFO=" and before the next ";" in the "INFO"
df['GENEINFO'] = df.INFO.str.extract(r'((?<=GENEINFO=).*)',\
                                      expand=False).str.replace(r'(;.*)','')

# the clinical significant is the value after the "CLNSIG=" and before the next ";" in the "INFO"
df['CLNSIG'] = df.INFO.str.extract(r'((?<=CLNSIG=).*)',\
                                      expand=False).str.replace(r'(;.*)','')

# the disease name is the value after the "CLNDN=" and before the next ";" in the "INFO"
df['CLNDN'] = df.INFO.str.extract(r'((?<=CLNDN=).*)',\
                                      expand=False).str.replace(r'(;.*)','')

# Mutation position: replace "O" by '0' and change data type to 'int' 
df['POS'] = df.POS.str.replace(r'([Oo])','0')
df = df.astype({'POS': 'int'})

# Change order of columns & rename
df = df[['GENEINFO','ID','CHROM','POS','REF','ALT','CLNSIG','CLNDN']]
df.columns = ['gene_name', 'mutation_ID', 'chromosome','mutation_position',
             'mutation_value_ref', 'mutation_value_alt','clinical_significant','disease_name']

# Replace missing values with "Not_Given"
df.fillna('Not_Given', inplace=True)

df

,gene_name,mutation_ID,chromosome,mutation_position,mutation_value_ref,mutation_value_alt,clinical_significant,disease_name
0,ISG15:9636,475283,1,1014042,G,A,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
1,ISG15:9636,542074,1,1014122,C,T,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
2,ISG15:9636,183381,1,1014143,C,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
3,ISG15:9636,542075,1,1014179,C,T,Uncertain_significance,Immunodeficiency_38_with_basal_ganglia_calcifi...
4,ISG15:9636,475278,1,1014217,C,T,Benign,Immunodeficiency_38_with_basal_ganglia_calcifi...
...,...,...,...,...,...,...,...,...
102316,PIK3CA:5290,403908,3,179210507,A,G,Uncertain_significance,Cowden_syndrome
102317,PIK3CA:5290,526648,3,179210511,T,C,Likely_benign,Cowden_syndrome
102318,PIK3CA:5290,526640,3,179210515,A,C,Uncertain_significance,Cowden_syndrome|Hereditary_cancer-predisposing...
102319,PIK3CA:5290,246681,3,179210516,A,G,Uncertain_significance,Cowden_syndrome


In [13]:
### Data Exploratory
# Print out all clinical significant values and counts for each value
print('Clinical significant values:\n')
print(df.clinical_significant.describe())
print('')
print(df.clinical_significant.value_counts())

Clinical significant values:

count                     102321
unique                        43
top       Uncertain_significance
freq                       47980
Name: clinical_significant, dtype: object

Uncertain_significance                                                                      47980
Likely_benign                                                                               17885
Pathogenic                                                                                  12313
Likely_pathogenic                                                                            6269
Benign                                                                                       6138
Conflicting_interpretations_of_pathogenicity                                                 5404
Benign/Likely_benign                                                                         3338
Not_Given                                                                                    1797
Pathogenic/

In [14]:
## Data Exploratory

# Clinical significant
print('Disease name values:\n')
print(df.disease_name.describe())
print('')
print(df.disease_name.value_counts().head(10))

print('\nGene Name:')
print(df.gene_name.describe())

print('\nMutation Value Reference:')
print(df.mutation_value_ref.describe())

print('\nMutation Value Alternate:')
print(df.mutation_value_alt.describe())

Disease name values:

count            102321
unique             6140
top       not_specified
freq              14400
Name: disease_name, dtype: object

not_specified                                                        14400
Not_Given                                                            12670
Hereditary_cancer-predisposing_syndrome                               2951
Limb-girdle_muscular_dystrophy,_type_2J|Dilated_cardiomyopathy_1G     2588
Lynch_syndrome                                                        1717
not_specified|not_provided                                            1190
Hereditary_nonpolyposis_colon_cancer                                  1095
Alstrom_syndrome                                                       846
Nemaline_myopathy_2                                                    769
Cardiovascular_phenotype                                               742
Name: disease_name, dtype: int64

Gene Name:
count                         102321
unique         

In [15]:
## Extract dangerous gene & mutation combination:
# Consider it's dangerous when containing "pathogenic" in the clinical significant 
df2 = df[df.clinical_significant.str.lower().str.contains('pathogenic')]

# And it does not contain "conflicting_interpretations_of_pathogenicity" 
df2 = df2[~df2.clinical_significant.str.lower().str.contains('pathogenicity')]

In [16]:
## Sanity Check after filtering:
print('Disease Name checking:')
print(df2.disease_name.describe())
print(df2.disease_name.value_counts().head(15))

print('\nGene_name without Not_Given entries:')
print(df2.gene_name[df2.gene_name == 'Not_Given'].count())

print('\nMutation_ID without Not_Given entries:')
print(df2.mutation_ID[df2.mutation_ID == 'Not_Given'].count())

print('\nChromosome without Not_Given entries:')
print(df2.chromosome[df2.chromosome == 'Not_Given'].count())

print('\nMutation position without Not_Given entries:')
print(df2.mutation_position[df2.mutation_position == 'Not_Given'].count())


Disease Name checking:
count         19474
unique         2976
top       Not_Given
freq           3712
Name: disease_name, dtype: object
Not_Given                                                 3712
Lynch_syndrome                                             788
Hereditary_cancer-predisposing_syndrome                    432
Ehlers-Danlos_syndrome,_type_4                             418
Primary_pulmonary_hypertension                             305
Nemaline_myopathy_2                                        234
Inborn_genetic_diseases                                    201
Severe_myoclonic_epilepsy_in_infancy                       190
Usher_syndrome,_type_2A|Retinitis_pigmentosa_39            181
Hereditary_nonpolyposis_colon_cancer                       178
Primary_hyperoxaluria,_type_I                              175
Primary_dilated_cardiomyopathy                             165
Alstrom_syndrome                                           156
Biotinidase_deficiency                      

In [17]:
## Remove "Not_Given" or "not_specified" from dataframe due to having no associated information
df2 = df2[~df2.disease_name.isin(['Not_Given', 'not_specified','.'])]
df2 = df2[~df2.gene_name.isin(['Not_Given','not_specified','.'])]
df2 = df2.reset_index()
df2 = df2.drop('index', axis=1)
print('\nClinical Significant checking:')
print(df2.clinical_significant.describe())
print(df2.clinical_significant.value_counts())
df2


Clinical Significant checking:
count          15750
unique            13
top       Pathogenic
freq           10418
Name: clinical_significant, dtype: object
Pathogenic                                   10418
Likely_pathogenic                             4458
Pathogenic/Likely_pathogenic                   836
Pathogenic,_risk_factor                         11
Pathogenic/Likely_pathogenic,_other              8
Pathogenic/Likely_pathogenic,_risk_factor        6
Likely_pathogenic,_risk_factor                   5
Pathogenic,_Affects                              2
Pathogenic,_other                                2
Pathogenic,_association,_protective              1
Likely_pathogenic,_association                   1
Likely_pathogenic,_other                         1
Pathogenic,_protective                           1
Name: clinical_significant, dtype: int64


,gene_name,mutation_ID,chromosome,mutation_position,mutation_value_ref,mutation_value_alt,clinical_significant,disease_name
0,ISG15:9636,183381,1,1014143,C,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
1,ISG15:9636,161455,1,1014316,C,CG,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
2,ISG15:9636,161454,1,1014359,G,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
3,AGRN:375790,243036,1,1022225,G,A,Pathogenic,Congenital_myasthenic_syndrome
4,AGRN:375790,243037,1,1022313,A,T,Pathogenic,Congenital_myasthenic_syndrome
...,...,...,...,...,...,...,...,...
15745,PIK3CA:5290,39706,3,179210289,TAGA,T,Pathogenic,Megalencephaly_cutis_marmorata_telangiectatica...
15746,PIK3CA:5290,376470,3,179210291,G,A,Likely_pathogenic,Hepatocellular_carcinoma|Transitional_cell_car...
15747,PIK3CA:5290,376471,3,179210291,G,C,Likely_pathogenic,Hepatocellular_carcinoma|Transitional_cell_car...
15748,PIK3CA:5290,45465,3,179210292,AAGATTTGCTGAACCC,A,Likely_pathogenic,Non-small_cell_lung_cancer


### My Assumptions:

The "INFOR" data does not does not follow a particular order, but the variables are separated by a ";" and a variable name and its values are separated by a "=" in between of them: variable_name=variable_value. We could extract the value for each variable using regular expression extract method.

Variables of interest:
- Gene name: the "GENEINFO" data in the "INFOR" column contains the gene symbol and the gene id. The gene symbol and id are separated by a colon (:), However, because there could be mutiple genes in an entry, therefore, I extract the complete data from the "GENEINFO" and do not separate between gene symbol & id. 
- Mutation ID number: is the "ID" column
- Mutation position: includes the chromosome "CHROM" column and position "POS" column
- Mutation value: includes reference base "REF" column and alternate base "ALT" column
- Clinical significant: is the "CLNSIG" variable in the "INFOR" column
- Disease that is implicated: is the "CLNDN" variable in the "INFOR" column

Other assumptions:
- Mutation position (POS): assume the position should be integer only, and some wrong entries with "O" should be replaced by '0' (zero).
- The mutation values for reference and alternative bases (REF & ALT) as listed in the dataset are correct, a letter could be used multiple times in the value and it does not need to be corrected.

Dangerous mutations:
- Assume the dangerous gene and mutation combination contains the following words in its clinical significant description: "pathogenic", "likely_pathogenic". 
- Any clinical significant having "conflicting_interpretations_of_pathogenicity", "risk_factor", "association", "affects" and not having the sigle word "pathogenic" will not be considered as dangerous mutation, due to being on debate and not confirmed as the causes of the associated diseases.
- Any record has "pathogenic" in the clinical significant but does not have data for the disease name, or gene name (or listed as "Not_Given") is excluded from the list of dangerous gene and mutation combination. Due to unclear disease or lacking of gene name information.

### Final Result

Below is the table for the first 100 harmful mutations to be present to my manager

In [18]:
## Print our the total number of harmful mutations and the first 100 harmful mutations
print('\nTotal of harmful mutations:', df2.shape[0])
df2.head(100)


Total of harmful mutations: 15750


,gene_name,mutation_ID,chromosome,mutation_position,mutation_value_ref,mutation_value_alt,clinical_significant,disease_name
0,ISG15:9636,183381,1,1014143,C,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
1,ISG15:9636,161455,1,1014316,C,CG,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
2,ISG15:9636,161454,1,1014359,G,T,Pathogenic,Immunodeficiency_38_with_basal_ganglia_calcifi...
3,AGRN:375790,243036,1,1022225,G,A,Pathogenic,Congenital_myasthenic_syndrome
4,AGRN:375790,243037,1,1022313,A,T,Pathogenic,Congenital_myasthenic_syndrome
5,AGRN:375790,574478,1,1041354,CGCCCGCCAGGAGAATGTCTTCAAGAAGTTCGACG,C,Pathogenic,"Myasthenic_syndrome,_congenital,_8"
6,AGRN:375790,126556,1,1041582,C,T,Pathogenic,Congenital_myasthenic_syndrome|Myasthenic_synd...
7,AGRN:375790,243038,1,1042136,T,TC,Pathogenic,Congenital_myasthenic_syndrome
8,AGRN:375790,243039,1,1050473,G,A,Pathogenic,Congenital_myasthenic_syndrome
9,AGRN:375790,18241,1,1050575,G,C,Pathogenic,Congenital_myasthenic_syndrome|Myasthenic_synd...
